# Fine-tuned Based Models for sentiment analysis

In [15]:
import joblib
import numpy as np
import pandas as pd

import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertConfig


from sklearn.metrics import classification_report, f1_score
import matplotlib.pyplot as plt

## Read Embeddings & labels

In [2]:
X_train = joblib.load('data/train_embeddings.pkl')
y_train = pd.read_csv('data/training.csv')['label'].values

X_val = joblib.load('data/validation_embeddings.pkl')
y_val = pd.read_csv('data/validation.csv')['label'].values

X_test = joblib.load('data/test_embeddings.pkl')
y_test = pd.read_csv('data/test.csv')['label'].values

In [3]:
print(X_train.shape, y_train.shape)
print(type(X_train), type(y_train))

(16000, 768) (16000,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


## Transform Dataframe into Dataloader

In [6]:
class EmotionDataset(Dataset): # defines how the text is pre-processed before sending it to the neural network.
    # generates 
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings  # Lista de embeddings generados por RoBERTa
        self.labels = labels.squeeze()  # Lista de etiquetas

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx): # []
        return torch.tensor(self.embeddings[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

In [11]:
# Dataset type conversion
train_dataset = EmotionDataset(X_train, y_train)
val_dataset = EmotionDataset(X_val, y_val)
test_dataset = EmotionDataset(X_test, y_test)

# will feed the data in batches to the neural network for suitable training and processing.
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(type(train_dataset), type(train_loader))

<class '__main__.EmotionDataset'> <class 'torch.utils.data.dataloader.DataLoader'>


## BERT model

Parameters:
- Input_size: Embedding length
- Heads: Number of heads

In [16]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 6)  # Adjust input size if needed
    
    def forward(self, embeddings):  # Only embeddings are required
        output_2 = self.l2(embeddings)
        output = self.l3(output_2)
        output = torch.argmax(output, dim=1)
        return output

In [17]:
device = 'cuda' if cuda.is_available() else 'cpu'

/home/luisdogo/miniconda3/envs/sentiment-deep_learning/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647378361/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [18]:
model = BERTClass()
model.to(device)

BERTClass(
  (l2): Dropout(p=0.3, inplace=False)
  (l3): Linear(in_features=768, out_features=6, bias=True)
)

In [19]:
def loss_fn(outputs, targets):
    print(f"Output shape: {outputs.shape}, Target shape: {targets.shape}")
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [23]:
LEARNING_RATE = .01

In [22]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

NameError: name 'LEARNING_RATE' is not defined

In [20]:
def train(epoch):
    model.train()
    for _, data in enumerate(train_loader, 0):
        # Unpack the data from the DataLoader
        embeddings, targets = data
        embeddings = embeddings.to(device, dtype=torch.float32)  # Move embeddings to the device
        targets = targets.to(device, dtype=torch.long)  # Move targets to the device

        # Forward pass through the model
        outputs = model(embeddings)

        # Calculate loss
        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        
        if _ % 5000 == 0:  # Print loss for every 5000 steps
            print(f'Epoch: {epoch}, Loss: {loss.item()}')
        
        # Backward pass and optimization step
        loss.backward()
        optimizer.step()


In [21]:
EPOCHS = 5
for epoch in range(EPOCHS):
    train(epoch)

NameError: name 'optimizer' is not defined

In [34]:
# pick base pre-trained model

# define feature extraction layer(docking layer to classification head), usually "bottle-neck layer"(last layer before flatten op)

#Feature extraction
#In this step, you will freeze the convolutional base created from the previous step and to use as a feature extractor
# #Additionally, you add a classifier on top of it and train the top-level classifier.


In [35]:
# define the classification head (class)



### Training & Evaluation

### Hyperparameters

In [36]:
# instantiate and evaluate 

### Visualization

In [ ]:
# training & Validation Loss
# Training & Validation F1-score
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


## RoBERTa model

In [ ]:
# will be done if we finish the bert model on time